In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
figer_path = base_dir + "FIGER/"

Mounted at /content/drive/


In [2]:
import json

with open(figer_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  1505765 elementi


In [4]:
#ci sono 36 frasi che hanno le mentions vuote, le cancello
indici = []
for i,elem in enumerate(examples_train):
  if len(elem['mentions'])==0:
     indici.append(i)

pop=0
for num in indici:
  del examples_train[num-pop]
  pop+=1

In [5]:
#examples_train=examples_train[0:10000]

In [5]:
entire_dataset_list = iter(examples_train)

In [6]:
def skip_this_phrase(mentions,dictionary):
    prod = 1
    for elem in mentions:
      prod*=len(elem['labels'])

    if prod<=10:
      dictionary['minori=_10']+=1
    if prod>10 and  prod<=20:
      dictionary['maggiori_10_minori=20']+=1
    if prod>20 and prod<=50:
      dictionary['maggiori_20_minori=50']+=1
    if prod>50 and prod<=100:
      dictionary['maggiori_50_minori=100']+=1
    if prod>100 and prod<=200:
      dictionary['maggiori_100_minori=200']+=1
    if (prod>200):
      dictionary['maggiori_200']+=1
    if (prod>20):
      return 0
    
    return prod


In [7]:
import operator
import itertools
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords

lunghezze={
    'minori=_10':0,
    'maggiori_10_minori=20':0,
    'maggiori_20_minori=50':0,
    'maggiori_50_minori=100':0,
    'maggiori_100_minori=200':0,
    'maggiori_200':0,
}

fp = open(path_corpus_data + 'corpus_data_entire_figer.txt', "w")
restricted_set = set()
#mean=0
for num,i in enumerate(tqdm(entire_dataset_list)):
  
  #calcolo le permutazioni in modo grossolano e skippo le frasi che genererebbero più di 200 permutazioni
  perm_priori=skip_this_phrase(i['mentions'],lunghezze)
  if (perm_priori==0):
    continue
  #else:
    #print(f"frase {num}) permutazioni a priori: {prod}")
  #mean+=prod
  istance = i['mentions'][:]
  if len(istance)>1:
    d = sorted(istance, key=operator.itemgetter("start"))
    d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}
    all_list=[]
    for key, value in d.items():
      all_list.append(value)

    permutations = list(itertools.product(*all_list))
    #print(f"{num}) Permutations: {permutations}")
    iter = 0
    real_perm = 0
    for p in permutations:
          mini_list = []
          for lista_labels in p:
            mini_list.append(lista_labels['labels'])
          
          permutations_2 = itertools.product(*mini_list)
          real_perm+=len(list(permutations_2))
          for p_2 in permutations_2:
            phrase = i['tokens'][:]
            num_pop=0
            for h in range(0, len(p_2)):
              start = permutations[iter][h]['start']
              end = permutations[iter][h]['end']
              phrase[start-num_pop:end-num_pop] = [p_2[h]]
              if ((end-start)>1):
                num_pop += (end - start) - 1
              restricted_set.add(p_2[h])

            fp.write((remove_stopwords(' '.join(phrase)) + '\n'))
          iter+=1
    if real_perm!=perm_priori:
      print(f"Frase {num}, permutazioni a priori {perm_priori}, permutazioni effettive {real_perm}")
  else:
    #print(f"Una sola mentions, permutazioni effettive {len(istance[0]['labels'])}")
    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(k)
      else:
        phrase[i['mentions'][0]['start']:i['mentions'][0]['end']] = [k]
        restricted_set.add(k)

      fp.write((remove_stopwords(' '.join(phrase)) + '\n'))

fp.close()

104216it [00:01, 57549.27it/s]

Frase 95886, permutazioni a priori 1, permutazioni effettive 2


445735it [00:07, 54799.17it/s]

Frase 440055, permutazioni a priori 8, permutazioni effettive 32


562984it [00:09, 58885.82it/s]

Frase 555593, permutazioni a priori 2, permutazioni effettive 4


1505729it [00:27, 55316.22it/s]


In [11]:
examples_train[5220]

{'fileid': 'wex/20110513/0/15/149',
 'mentions': [{'end': 14,
   'labels': ['/person/author', '/person'],
   'start': 12},
  {'end': 25, 'labels': ['/location/city', '/location'], 'start': 24}],
 'senid': 14,
 'tokens': ['October',
  '-',
  'George',
  'Stephenson',
  "'s",
  'steam',
  'locomotive',
  ',',
  'The',
  'Rocket',
  ',',
  'defeats',
  'John',
  'Ericsson',
  "'s",
  'Novelty',
  'and',
  'thus',
  'wins',
  'The',
  'Rainhill',
  'Trials',
  'held',
  'near',
  'Liverpool',
  '.']}

In [10]:
lunghezze

{'maggiori_100_minori=200': 0,
 'maggiori_10_minori=20': 1,
 'maggiori_200': 0,
 'maggiori_20_minori=50': 1,
 'maggiori_50_minori=100': 0,
 'minori=_10': 8}

In [11]:
print(f"Frasi skippate {lunghezze['maggiori_200']}, frasi processate {len(examples_train)-lunghezze['maggiori_200']}")

Frasi skippate 0, frasi processate 10


In [ ]:
restricted_list = list(restricted_set)

In [ ]:
import pickle
with open(path_corpus_data + 'restricted_list_figer.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)